### Week 5 Day 4

AutoGen Core - **Distributed**

I'm only going to give a Teaser of this!!

Partly because I'm unsure how relevant it is to you. If you'd like me to add more content for this, please do let me know..

In [ ]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = True #one run with this set to True and another with this set to False
#when set to FALSE, later on three separate runtimes are created and agents are interacting between runtimes

### Start with our Message class

In [2]:

@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime

In [ ]:
"""
uses grpc instead of single thread.

gRPC is an open-source, high-performance framework for remote procedure calls, allowing services to communicate efficiently 
across data centers or in the "last mile" to devices and browsers. It uses Protocol Buffers to serialize data, enabling it to 
work across many languages and platforms. gRPC is built on HTTP/2 and supports streaming, making it suitable for modern 
microservices and distributed applications. 

grpc is a cross-language approach for sending function call across different language and process boundaries
"""

from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() #so now the local host is running

### Let's reintroduce a tool

In [4]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [5]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

### And make some Agents

In [14]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True) # we pass the serper tool here

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent): #using gpt-4o-mini for both, we don't actually need player1 and player2 here... we can just do with one player
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1) 
        response2 = await self.send_message(message2, inner_2)
        """ 
        # These calls look exactly the same as standard/single threading but now this is going to be happening using 
        # gRPC remotely orchestrated by this distributed runtime. But that is completely unknown to us as far as we're concerned.
        # We're just doing exactly the same thing. And that is the power of Autogen core distributed that we don't have 
        # to worry about the fact that these are different processes running, and they could be written in different 
        # computer programming languages. And all of the stitching together of messages is happening for us, just based on 
        # looking up player one and player two.
        """

        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [7]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




In [8]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [9]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are some reasons in favor of using AutoGen for a new AI Agent project:

1. **Multi-agent Coordination**: AutoGen excels at managing and coordinating multiple AI agents, enabling them to collaborate effectively towards shared goals. This is particularly beneficial for complex tasks that require distributed processing.

2. **Flexible Design**: AutoGen offers both code-based and graphical user interface (GUI) options through AutoGen Studio, allowing for rapid prototyping and deployment. This flexibility can speed up development and make it accessible to a broader range of developers.

3. **Scalable Solutions**: AutoGen can scale seamlessly, adapting to varying project sizes and complexities, making it suitable for both small and large applications.

4. **Enhanced Collaboration**: It facilitates the interaction between different AI agents, resulting in improved outcomes through cooperative strategies.

5. **User-Friendly Environment**: The availability of a GUI can help teams not fully versed in programming to contribute effectively, promoting a more inclusive development environment.

These advantages make AutoGen a compelling choice for projects involving AI agents. 

TERMINATE

## Cons of AutoGen:
Here are some reasons against choosing AutoGen for your AI Agent project:

1. **High Cost Structure**: AutoGen operates on a user-based pricing model, which can become quite expensive, especially for larger teams.

2. **Import Function Issues**: Users have reported problems with PDF uploads, including formatting issues that can hinder functionality and usability.

3. **Magic Feature Limitations**: The AI's suggestions may not always be reliable, with responses frequently being incorrect or not meeting user expectations.

These drawbacks should be carefully considered when evaluating whether to implement AutoGen in your project. TERMINATE



## Decision:

Based on the compelling pros and notable cons of AutoGen, I recommend moving forward with its implementation for the AI Agent project. The strengths in multi-agent coordination, flexibility in design, and scalability align well with the project's potential complexities and collaborative requirements. While cost and some functionality issues are valid concerns, the advantages appear to outweigh the drawbacks, especially if effective budget management and operational adjustments are put in place to address them. In conclusion, AutoGen presents a valuable opportunity for enhancing project outcomes and team efficiency.

TERMINATE

In [10]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [11]:
await host.stop()